In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
import psycopg2
from settings import DB_PARAMS
from settings import env_path
from dotenv import load_dotenv
from google import genai
from google.genai import types
load_dotenv(env_path)


In [ ]:
from langchain_teddynote import logging
# input your project name
logging.langsmith("Test_Demo_App")

In [ ]:
from utils import init_llm

llm = init_llm()
model_name = "BAAI/bge-m3"

### Build StateGraph

1. 프로젝트 목적에 맞는 상태 그래프
2. google 검색하는 tool


In [ ]:
def get_db_connection():
    return psycopg2.connect(**DB_PARAMS)

def embed_text(text, model):
    return model.encode(text)



def search_metadata(
    vector,
    conn,
    k=3,
    ):
    """pgvector에서 관련성 높은 문서를 k개 검색합니다."""
    with conn.cursor() as cursor:
        # 벡터 검색 시 필요한 모든 컬럼을 가져옵니다.
        cursor.execute(
            """SELECT content,school_level, grade, domain, category
                FROM curriculum
                ORDER BY embedding <=> %s::vector LIMIT %s""",
            (list(vector), k)
        )
        # 결과를 딕셔너리 리스트로 변환
        results = [
            dict(zip([desc[0] for desc in cursor.description], row))
            for row in cursor.fetchall()
        ]
        return results

In [ ]:
from pydantic import BaseModel,Field
from typing import List,Literal,Optional
from typing import TypedDict
from typing import Dict,Any
# 노드 1의 출력 구조를 정의할 Pydantic 모델
class Requirements(BaseModel):
    """사용자 프롬프트에서 추출된 요구사항"""
    school_level: str = Field(description="학교급 (예: 초등학교, 중학교, 고등학교)")
    grade: str = Field(description="학년 (예: 1학년, 2학년)")
    subject: str = Field(description="과목 (예: 수학, 과학)")
    content_requests: List[Literal["학습 목표 생성", "문제 생성"]] = Field(description="요청된 콘텐츠 유형 목록")
    domain: str = Field(description="핵심 학습 주제 또는 단원명")
# 그래프 전체에서 공유될 상태 객체
class GraphState(TypedDict):
    prompt: str  # 사용자 초기 입력
    requirements: Optional[Requirements]  # 노드 1의 결과 (추출된 요구사항)
    retrieved_docs: Optional[List[dict]]  # 노드 2의 결과 (벡터 DB 검색 결과)
    generated_learning_goals: Optional[str]  # 노드 3의 결과 (학습 목표)
    generated_problems: Optional[str]  # 노드 3의 결과 (문제)
    final_response: Optional[str]  # 최종 답변

In [ ]:
db_conn = get_db_connection()
client = genai.Client()

In [ ]:
# Define nodes
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from sentence_transformers import SentenceTransformer
def extract_requirements_node(state: GraphState) -> dict:
    """
    (노드 1) 사용자 프롬프트에서 요구사항과 메타데이터를 추출합니다.
    """
    prompt = state["prompt"]

    # LLM이 Pydantic 모델(Requirements)에 맞춰 구조화된 결과를 출력하도록 설정
    structured_llm = llm.with_structured_output(Requirements)
    # Pydantic 모델에 정의된 Literal 값을 가져옵니다.
    allowed_requests = Requirements.model_fields['content_requests'].annotation.__args__[0].__args__
    # 결과: ('학습 목표 생성', '문제 생성')

    system_prompt = f"""You are an expert at analyzing user requests for educational content creation.
    Extract the required information from the user's prompt.

    For the `content_requests` field, you MUST choose one or more values from the following exact list:
    {list(allowed_requests)}

    Do not use similar words or variations. For example, if the user asks for '연습 문제' or '퀴즈', you must map it to '문제 생성'. If they ask for '학습 목표'
    you must map it to '학습 목표 생성'.
    """

    # system_prompt = "You are an expert at analyzing user requests for educational content creation. Extract the required information from the user's prompt."
    user_prompt = f"다음 사용자 프롬프트에서 핵심 요구사항을 추출해줘: '{prompt}'"
    
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", user_prompt),
    ])
    
    chain = prompt_template | structured_llm
    extracted = chain.invoke({"prompt": prompt})
    
    print(f"-> 추출된 요구사항: {extracted}")

    return {"requirements": extracted}



def retrieve_from_db_node(state: GraphState) -> dict:
    """
    (노드 2) 추출된 요구사항을 기반으로 벡터 DB에서 관련 메타데이터를 검색합니다.
    """
    print("\n--- 2. 벡터 DB 검색 노드 실행 ---")
    requirements = state["requirements"]
    school_level = requirements.school_level
    grade = requirements.grade
    domain = requirements.domain
    subject = requirements.subject
    
    concated_metadata = f"{school_level} {subject} {grade} {domain}"
    model = SentenceTransformer(model_name)     
    embed_metadata = model.encode(concated_metadata)
    # embed_metadata = client.models.embed_content(
    #     model="gemini-embedding-001",
    #     contents=concated_metadata,
    #     config=types.EmbedContentConfig(output_dimensionality=1024)
    # ).embeddings[0].values
    
    # 가상 벡터 DB에서 검색
    retrieved_data = search_metadata(
        vector=embed_metadata,
        conn=db_conn
    )
    print(retrieved_data)
    return {"retrieved_docs": retrieved_data}

def consolidate_response_node(state: GraphState) -> dict:
    """
    (노드 4) 병렬로 생성된 콘텐츠들을 종합하여 최종 답변을 만듭니다.
    """
    print("\n--- 4. 최종 답변 종합 노드 실행 ---")
    requirements = state["requirements"]
    learning_goals = state.get("generated_learning_goals")
    problems = state.get("generated_problems")

    final_response_parts = []
    final_response_parts.append(
        f"요청하신 **{requirements.school_level} {requirements.grade} {requirements.subject} - '{requirements.domain}'** 단원에 대한 콘텐츠입니다.\n"
    )
    # print(f"learning_goals : {learning_goals}")
    # print(f"problems : {problems}")
    
    if learning_goals:
        final_response_parts.append("### 학습 목표\n" + learning_goals.content)
    
    if problems:
        final_response_parts.append("\n### 연습 문제\n" + problems.content)
        
    final_response = "\n".join(final_response_parts)
    print("-> 최종 답변 생성 완료")
    
    return {"final_response": final_response}

In [ ]:

from langchain_core.tools import tool

def generate_learning_goals_node(state: GraphState) -> dict:
    """
    입력된 정보를 바탕으로 학습 목표를 생성합니다.
    """
    requirements = state["requirements"]
    docs = state["retrieved_docs"]

    prompt_template = ChatPromptTemplate.from_template(
        "{school_level} {grade} {subject} 과목의 '{domain}' 단원에 대한 학습 목표를 생성해줘.\n"
        "참고 자료:\n{docs}"
    )
    
    chain = prompt_template | llm
    result = chain.invoke({
        "school_level": requirements.school_level,
        "grade": requirements.grade,
        "subject": requirements.subject,
        "domain": requirements.domain,
        "docs": docs
    })

    return {"generated_learning_goals": result}

def generate_problems_node(state: GraphState) -> dict:
    """
    (노드 3-2) 검색된 정보를 바탕으로 연습 문제를 생성합니다. (병렬 처리 대상)
    """
    requirements = state["requirements"]
    docs = state["retrieved_docs"]

    prompt_template = ChatPromptTemplate.from_template(
        "{school_level} {grade} {subject} 과목의 '{domain}' 단원에 대한 연습 문제를 2개 생성해줘.\n"
        "참고 자료:\n{docs}"
    )
    
    chain = prompt_template | llm
    result = chain.invoke({
        "school_level": requirements.school_level,
        "grade": requirements.grade,
        "subject": requirements.subject,
        "domain": requirements.domain,
        "docs": docs
    })

    print(f"-> 생성된 문제: {result}")
    return {"generated_problems": result}




In [ ]:
# 3. 엣지(Edge)를 위한 조건부 라우팅 함수 정의
# -------------------------------------------------------------------

def route_content_generation(state: GraphState) -> List[str]:
    """
    사용자의 요청에 따라 다음에 실행할 콘텐츠 생성 노드를 결정합니다.
    - 리스트를 반환하여 여러 노드를 병렬로 실행시킬 수 있습니다.
    """
    print("\n--- 라우팅: 생성할 콘텐츠 결정 ---")
    content_requests = state["requirements"].content_requests
    
    next_nodes = []
    if "학습 목표 생성" in content_requests:
        next_nodes.append("generate_learning_goals")
    if "문제 생성" in content_requests:
        next_nodes.append("generate_problems")
    
    print(f"-> 다음 노드(병렬 실행): {next_nodes}")
    return next_nodes

In [ ]:
# 4. 그래프(Graph) 생성 및 엣지 연결
# -------------------------------------------------------------------

# 그래프 객체 생성
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("extract_requirements", extract_requirements_node)
workflow.add_node("retrieve_from_db", retrieve_from_db_node)
workflow.add_node("generate_learning_goals", generate_learning_goals_node)
workflow.add_node("generate_problems", generate_problems_node)
workflow.add_node("consolidate_response", consolidate_response_node)

# 엣지 연결
workflow.set_entry_point("extract_requirements")
workflow.add_edge("extract_requirements", "retrieve_from_db")
workflow.add_edge("generate_learning_goals", "consolidate_response")
workflow.add_edge("generate_problems", "consolidate_response")

# 조건부 엣지: 검색 노드 이후, 라우팅 함수 결과에 따라 병렬 노드로 분기
workflow.add_conditional_edges(
    "retrieve_from_db",
    route_content_generation,
    # 라우팅 함수가 반환한 노드 이름 리스트에 포함된 각 노드는
    # 모두 'consolidate_response' 노드로 연결됩니다.
    # LangGraph는 이 지점에서 병렬 실행을 처리하고, 모든 병렬 작업이
    # 완료된 후에야 다음 노드(consolidate_response)를 실행합니다.
    {
        "generate_learning_goals": "generate_learning_goals",
        "generate_problems": "generate_problems",
    }
)

workflow.add_edge("consolidate_response", END)

# 그래프 컴파일
app = workflow.compile()

In [ ]:
from langchain_teddynote.graphs import visualize_graph
visualize_graph(app)

In [ ]:
# 실행할 프롬프트
import numpy as np
from psycopg2.extensions import register_adapter, AsIs

# numpy.float32 타입을 Python float으로 변환하도록 등록
register_adapter(np.float32, lambda a: AsIs(a.item())) 
user_prompt = "고등학교 1학년 수학 과목의 '경우의 수' 단원의 집합 과목에 대한 학습 목표랑 연습 문제를 만들어 주세요."

# 입력값 설정
inputs = {"prompt": user_prompt}

# 그래프 실행 및 결과 확인
final_state = app.invoke(inputs)

print("\n\n================ 최종 결과 ================")
print(final_state['final_response'])